## Grab SafeGraph data from AWS 

#### code from ryan@safegraph.com

In [10]:
! pwd

/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/notebooks


In [11]:
#Configure the aws profile **check the slack for new access keys weekly!
#! aws configure --profile safegraph
#Downloads all current available data: *takes a while to run
! aws s3 sync s3://sg-c19-response/social-distancing/v2/ /Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/SafeGraph_data --profile safegraph

download: s3://sg-c19-response/social-distancing/v2/2020/01/01/2020-01-01-social-distancing.csv.gz to ../SafeGraph_data/2020/01/01/2020-01-01-social-distancing.csv.gz
download: s3://sg-c19-response/social-distancing/v2/2020/01/03/2020-01-03-social-distancing.csv.gz to ../SafeGraph_data/2020/01/03/2020-01-03-social-distancing.csv.gz
download: s3://sg-c19-response/social-distancing/v2/2020/01/02/2020-01-02-social-distancing.csv.gz to ../SafeGraph_data/2020/01/02/2020-01-02-social-distancing.csv.gz
download: s3://sg-c19-response/social-distancing/v2/2020/01/05/2020-01-05-social-distancing.csv.gz to ../SafeGraph_data/2020/01/05/2020-01-05-social-distancing.csv.gz
download: s3://sg-c19-response/social-distancing/v2/2020/01/04/2020-01-04-social-distancing.csv.gz to ../SafeGraph_data/2020/01/04/2020-01-04-social-distancing.csv.gz
download: s3://sg-c19-response/social-distancing/v2/2020/01/06/2020-01-06-social-distancing.csv.gz to ../SafeGraph_data/2020/01/06/2020-01-06-social-distancing.csv.g

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [6]:
sdm_spark_raw = pd.read_csv("s3://sg-c19-response/social-distancing/v2/2020/*/*/*.csv.gz", header=True, escape="\"", inferSchema=False)

NameError: name 'spark' is not defined